# Champion Attribute Analysis

Explore computed champion attributes and validate the mathematical models.

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load computed attributes
with open('../data/processed/computed_attributes.json', 'r') as f:
    data = json.load(f)

# Convert to DataFrame
rows = []
for champ_id, attrs in data['champions'].items():
    row = {'champion': champ_id}
    row.update({k: v for k, v in attrs.items() if k != 'range_profile'})
    row['aa_range'] = attrs['range_profile']['auto_attack']
    row['ability_range'] = attrs['range_profile']['effective_ability']
    rows.append(row)

df = pd.DataFrame(rows)
print(f"Loaded {len(df)} champions")
df.head()

In [ ]:
# Summary statistics
numeric_cols = ['cc_score', 'mobility_score', 'survivability_late', 
                'waveclear_speed', 'gold_dependency', 'aoe_capability']
df[numeric_cols].describe()

In [ ]:
# Scatter plot: Mobility vs Survivability
plt.figure(figsize=(12, 8))
plt.scatter(df['mobility_score'], df['survivability_late'], alpha=0.6)

# Annotate interesting champions
champions_to_label = ['Zed', 'Malphite', 'Jinx', 'Kassadin', 'Lulu']
for champ in champions_to_label:
    row = df[df['champion'] == champ].iloc[0]
    plt.annotate(champ, (row['mobility_score'], row['survivability_late']))

plt.xlabel('Mobility Score')
plt.ylabel('Survivability (Late Game)')
plt.title('Champion Mobility vs Survivability')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Distribution of damage patterns
pattern_counts = df['damage_pattern'].value_counts()
print("Damage Pattern Distribution:")
print(pattern_counts)

plt.figure(figsize=(8, 5))
pattern_counts.plot(kind='bar')
plt.title('Distribution of Damage Patterns')
plt.xlabel('Pattern')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Top 10 most mobile champions
top_mobile = df.nlargest(10, 'mobility_score')[['champion', 'mobility_score']]
print("Top 10 Most Mobile Champions:")
print(top_mobile)

In [ ]:
# Top 10 highest CC champions
top_cc = df.nlargest(10, 'cc_score')[['champion', 'cc_score']]
print("Top 10 Highest CC Champions:")
print(top_cc)

In [ ]:
# Range analysis: Melee vs Ranged
df['is_melee'] = df['aa_range'] <= 200
print(f"Melee champions: {df['is_melee'].sum()}")
print(f"Ranged champions: {(~df['is_melee']).sum()}")

# Compare attributes
comparison = df.groupby('is_melee')[numeric_cols].mean()
print("\nMelee vs Ranged Attribute Comparison:")
print(comparison)

In [ ]:
# Correlation heatmap
plt.figure(figsize=(10, 8))
correlation = df[numeric_cols].corr()
sns.heatmap(correlation, annot=True, cmap='coolwarm', center=0)
plt.title('Attribute Correlation Matrix')
plt.tight_layout()
plt.show()